In [51]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.preprocessing import KBinsDiscretizer
import math
import warnings

In [78]:
cna= pd.read_csv ('after_mrmr/after_mrmr_CNA.csv')# will import datset

label= pd.read_csv ('after_mrmr/after_mrmr_label_all.csv')# will import datset

clinical= pd.read_csv ('after_mrmr/after_mrmr_clinical_all.csv')





KeyboardInterrupt: 

In [124]:
gene= pd.read_csv ('after_mrmr/after_mrmr_geneexp.csv')

In [91]:
cna=cna.rename(columns={'Unnamed: 0': 'Patient ID'})
cna=cna.rename(columns={'Patient ID': 'Patient_ID'})

In [92]:
cna

,Patient_ID,DDX11,KMO,LINC00229,DBF4,SSSCA1,DBR1,CYP7A1,CYBA,PGLS,...,LINC00358,SPDYE8P,SUCLG2,PGA5,LINC00515,LINC00659,LINC00299,SULT4A1,LINC00305,PCSK4
0,MB-0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,MB-0039,1.0,0.0,0.0,-1.0,-1.0,0.0,0.0,0.0,0.0,...,-1.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,MB-0045,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,...,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0
3,MB-0046,-1.0,1.0,-1.0,0.0,0.0,0.0,0.0,-1.0,-1.0,...,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0
4,MB-0048,0.0,0.0,-1.0,0.0,1.0,0.0,1.0,-1.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2168,MB-6020,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,-1.0,0.0,0.0
2169,MB-6213,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2170,MB-6230,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,...,0.0,-2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2171,MB-7148,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,...,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [114]:
label=label.rename(columns={'Patient ID': 'Patient_ID'})

In [115]:
label

,Patient_ID,OS_MONTHS
0,MB-0000,0.0
1,MB-0002,0.0
2,MB-0005,1.0
3,MB-0006,1.0
4,MB-0008,0.0
...,...,...
2501,MTS-T2425,0.0
2502,MTS-T2426,0.0
2503,MTS-T2427,0.0
2504,MTS-T2428,0.0


In [85]:
clinical=clinical.drop(columns=['Unnamed: 0'])

In [86]:
clinical=clinical.rename(columns={'PATIENT_ID': 'Patient_ID'})

In [87]:
clinical

,Patient_ID,LYMPH_NODES_EXAMINED_POSITIVE,NPI,CELLULARITY,CHEMOTHERAPY,COHORT,ER_IHC,HER2_SNP6,HORMONE_THERAPY,INFERRED_MENOPAUSAL_STATE,INTCLUST,AGE_AT_DIAGNOSIS,OS_STATUS,CLAUDIN_SUBTYPE,THREEGENE,VITAL_STATUS,LATERALITY,RADIO_THERAPY,HISTOLOGICAL_SUBTYPE,BREAST_SURGERY
0,MB-0000,10.0,6.044,NaN,0.0,1.0,1.0,1,1.0,1.0,4.0,75.65,1.0,3.0,2.0,1.0,0.0,1.0,0.0,0.0
1,MB-0002,0.0,4.020,3.0,0.0,1.0,1.0,1,1.0,0.0,4.0,43.19,1.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0
2,MB-0005,1.0,4.030,3.0,1.0,1.0,1.0,1,1.0,0.0,3.0,48.87,0.0,1.0,NaN,0.0,0.0,0.0,0.0,0.0
3,MB-0006,3.0,4.050,2.0,1.0,1.0,1.0,1,1.0,0.0,9.0,47.68,1.0,1.0,NaN,1.0,0.0,1.0,1.0,0.0
4,MB-0008,8.0,6.080,3.0,1.0,1.0,1.0,1,1.0,1.0,9.0,76.97,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2504,MTS-T2428,0.0,2.540,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,70.05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2505,MTS-T2429,0.0,4.560,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,63.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2506,MTS-T2430,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2507,MTS-T2431,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [127]:
gene

,Patient_ID,VPS13D,CF887450,T65755,USP3,ANK1,B3GALT4,TRAF1,BM983985,NR2C2AP,...,MPP5,EIF4E,LOC340073,AKAP2,AP2A1,AX747098,FKBP7,SLC7A6OS,ING4,HHATL
0,MB-0362,1.0,0.0,0.0,1.0,0.0,1.0,2.0,0.0,1.0,...,2.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0
1,MB-0346,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0
2,MB-0386,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,...,2.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0
3,MB-0574,1.0,0.0,0.0,1.0,0.0,1.0,2.0,1.0,1.0,...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0
4,MB-0503,1.0,0.0,0.0,1.0,0.0,1.0,2.0,1.0,1.0,...,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1899,MB-5465,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,...,1.0,0.0,1.0,2.0,1.0,2.0,1.0,2.0,1.0,0.0
1900,MB-5453,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
1901,MB-5471,1.0,0.0,0.0,1.0,0.0,1.0,2.0,0.0,1.0,...,2.0,0.0,1.0,2.0,1.0,1.0,1.0,0.0,1.0,0.0
1902,MB-5127,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,...,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0


In [126]:
gene=gene.rename(columns={'Unnamed: 0': 'Patient_ID'})

In [137]:
gene_join=gene.iloc[0:,0:1]

In [116]:
label.set_index('Patient_ID')

,OS_MONTHS
Patient_ID,
MB-0000,0.0
MB-0002,0.0
MB-0005,1.0
MB-0006,1.0
MB-0008,0.0
...,...
MTS-T2425,0.0
MTS-T2426,0.0
MTS-T2427,0.0


In [141]:
cna_1904=gene_join.set_index('Patient_ID').join(cna.set_index('Patient_ID'))
cna_1904.fillna(0.001)
cna_1904

,DDX11,KMO,LINC00229,DBF4,SSSCA1,DBR1,CYP7A1,CYBA,PGLS,CTNNA2,...,LINC00358,SPDYE8P,SUCLG2,PGA5,LINC00515,LINC00659,LINC00299,SULT4A1,LINC00305,PCSK4
Patient_ID,,,,,,,,,,,,,,,,,,,,,
MB-0362,1.0,2.0,-1.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,...,0.0,0.0,0.0,-1.0,0.0,1.0,0.0,-1.0,0.0,0.0
MB-0346,1.0,1.0,0.0,-1.0,1.0,-1.0,1.0,1.0,-1.0,0.0,...,-1.0,1.0,-1.0,1.0,2.0,1.0,1.0,0.0,-1.0,1.0
MB-0386,0.0,1.0,-1.0,1.0,0.0,0.0,2.0,-1.0,1.0,1.0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0
MB-0574,0.0,2.0,-1.0,-1.0,-1.0,2.0,2.0,0.0,0.0,-1.0,...,1.0,-1.0,-1.0,-1.0,0.0,1.0,-1.0,-1.0,-1.0,0.0
MB-0503,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,...,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MB-5465,-1.0,1.0,1.0,0.0,0.0,1.0,-1.0,0.0,0.0,1.0,...,0.0,0.0,-1.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0
MB-5453,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
MB-5471,0.0,2.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,...,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-1.0,0.0


In [142]:
label_1904=gene_join.set_index('Patient_ID').join(label.set_index('Patient_ID'))
label_1904.fillna(0.001)
label_1904

,OS_MONTHS
Patient_ID,
MB-0362,0.0
MB-0346,0.0
MB-0386,0.0
MB-0574,0.0
MB-0503,0.0
...,...
MB-5465,0.0
MB-5453,0.0
MB-5471,1.0


In [144]:
clinical_1904=gene_join.set_index('Patient_ID').join(clinical.set_index('Patient_ID'))
clinical_1904.fillna(0.001)
clinical_1904

,LYMPH_NODES_EXAMINED_POSITIVE,NPI,CELLULARITY,CHEMOTHERAPY,COHORT,ER_IHC,HER2_SNP6,HORMONE_THERAPY,INFERRED_MENOPAUSAL_STATE,INTCLUST,AGE_AT_DIAGNOSIS,OS_STATUS,CLAUDIN_SUBTYPE,THREEGENE,VITAL_STATUS,LATERALITY,RADIO_THERAPY,HISTOLOGICAL_SUBTYPE,BREAST_SURGERY
Patient_ID,,,,,,,,,,,,,,,,,,,
MB-0362,1.0,4.036,3.0,1.0,1.0,1.0,1,1.0,1.0,2.0,52.79,0.0,2.0,0.0,0.0,1.0,1.0,0.0,0.0
MB-0346,5.0,6.032,3.0,0.0,1.0,0.0,2,0.0,0.0,5.0,32.61,0.0,5.0,3.0,0.0,1.0,0.0,0.0,0.0
MB-0386,8.0,6.030,2.0,0.0,1.0,1.0,0,1.0,1.0,9.0,66.83,1.0,2.0,1.0,1.0,0.0,1.0,0.0,1.0
MB-0574,1.0,5.042,3.0,0.0,1.0,1.0,1,1.0,1.0,2.0,71.21,1.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0
MB-0503,0.0,3.046,3.0,0.0,1.0,1.0,2,1.0,1.0,7.0,60.77,1.0,2.0,1.0,1.0,1.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MB-5465,0.0,4.050,1.0,0.0,3.0,0.0,0,1.0,1.0,10.0,69.83,0.0,4.0,2.0,0.0,0.0,1.0,0.0,1.0
MB-5453,3.0,5.060,1.0,1.0,3.0,0.0,1,0.0,0.0,4.0,36.99,0.0,0.0,2.0,0.0,1.0,1.0,0.0,1.0
MB-5471,0.0,4.026,2.0,0.0,3.0,1.0,1,0.0,1.0,3.0,67.54,1.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0


In [64]:
clinical.shape

(2509, 20)

In [145]:
clinical_1904.shape

(1904, 19)

In [65]:
cna.shape

(2173, 201)

In [146]:
cna_1904.shape

(1904, 200)

In [66]:
label.shape

(2506, 2)

In [147]:
label_1904.shape

(1904, 1)

In [128]:
gene.shape

(1904, 401)

In [148]:
clinical_1904.to_csv(r'clinical_1904.csv', index = False)
cna_1904.to_csv(r'cna_1904.csv', index = False)
label_1904.to_csv(r'label_1904.csv', index = False)




In [156]:
merg1=clinical_1904.merge(cna_1904, left_on='Patient_ID', right_on='Patient_ID')
merg1

,LYMPH_NODES_EXAMINED_POSITIVE,NPI,CELLULARITY,CHEMOTHERAPY,COHORT,ER_IHC,HER2_SNP6,HORMONE_THERAPY,INFERRED_MENOPAUSAL_STATE,INTCLUST,...,LINC00358,SPDYE8P,SUCLG2,PGA5,LINC00515,LINC00659,LINC00299,SULT4A1,LINC00305,PCSK4
Patient_ID,,,,,,,,,,,,,,,,,,,,,
MB-0362,1.0,4.036,3.0,1.0,1.0,1.0,1,1.0,1.0,2.0,...,0.0,0.0,0.0,-1.0,0.0,1.0,0.0,-1.0,0.0,0.0
MB-0346,5.0,6.032,3.0,0.0,1.0,0.0,2,0.0,0.0,5.0,...,-1.0,1.0,-1.0,1.0,2.0,1.0,1.0,0.0,-1.0,1.0
MB-0386,8.0,6.030,2.0,0.0,1.0,1.0,0,1.0,1.0,9.0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0
MB-0574,1.0,5.042,3.0,0.0,1.0,1.0,1,1.0,1.0,2.0,...,1.0,-1.0,-1.0,-1.0,0.0,1.0,-1.0,-1.0,-1.0,0.0
MB-0503,0.0,3.046,3.0,0.0,1.0,1.0,2,1.0,1.0,7.0,...,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MB-5465,0.0,4.050,1.0,0.0,3.0,0.0,0,1.0,1.0,10.0,...,0.0,0.0,-1.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0
MB-5453,3.0,5.060,1.0,1.0,3.0,0.0,1,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
MB-5471,0.0,4.026,2.0,0.0,3.0,1.0,1,0.0,1.0,3.0,...,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,-1.0,0.0


In [160]:
Early_fusion2=merg1.merge(gene, left_on='Patient_ID', right_on='Patient_ID')
Early_fusion2=Early_fusion.set_index('Patient_ID')
Early_fusion2



,LYMPH_NODES_EXAMINED_POSITIVE,NPI,CELLULARITY,CHEMOTHERAPY,COHORT,ER_IHC,HER2_SNP6,HORMONE_THERAPY,INFERRED_MENOPAUSAL_STATE,INTCLUST,...,MPP5,EIF4E,LOC340073,AKAP2,AP2A1,AX747098,FKBP7,SLC7A6OS,ING4,HHATL
Patient_ID,,,,,,,,,,,,,,,,,,,,,
MB-0362,1.0,4.036,3.0,1.0,1.0,1.0,1,1.0,1.0,2.0,...,2.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0
MB-0346,5.0,6.032,3.0,0.0,1.0,0.0,2,0.0,0.0,5.0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0
MB-0386,8.0,6.030,2.0,0.0,1.0,1.0,0,1.0,1.0,9.0,...,2.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0
MB-0574,1.0,5.042,3.0,0.0,1.0,1.0,1,1.0,1.0,2.0,...,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0
MB-0503,0.0,3.046,3.0,0.0,1.0,1.0,2,1.0,1.0,7.0,...,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MB-5465,0.0,4.050,1.0,0.0,3.0,0.0,0,1.0,1.0,10.0,...,1.0,0.0,1.0,2.0,1.0,2.0,1.0,2.0,1.0,0.0
MB-5453,3.0,5.060,1.0,1.0,3.0,0.0,1,0.0,0.0,4.0,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
MB-5471,0.0,4.026,2.0,0.0,3.0,1.0,1,0.0,1.0,3.0,...,2.0,0.0,1.0,2.0,1.0,1.0,1.0,0.0,1.0,0.0


In [161]:
Early_fusion2.to_csv(r'for_fusion.csv', index = False)

